In [138]:
import csv
import folium 

In [139]:
gs_coords = (37.23076314126423, -80.42473322034601)
map = folium.Map(location=gs_coords, zoom_start=13, max_zoom=25)
folium.Marker(gs_coords, tooltip='Whittemore Roof', popup='Whittemore Roof').add_to(map)

In [140]:
with open("4-8-log.csv", "r") as f:
    reader = csv.reader(f, delimiter=",")
    data = list(reader)

In [141]:
locations = {"Drone Field": [], "Prices Fork": [], "Bald Knob": []}
miss_rate = {"Drone Field": [], "Prices Fork": [], "Bald Knob": []}
current_location = None
recv_format = {data[2][x]: x for x in range(len(data[2]))}
gps_format = {data[3][x]: x for x in range(len(data[3]))}
last_coords=None
for line in data[4:]:
    if "Drone Field" in line[0]:
        current_location = "Drone Field"
    elif "Prices Fork" in line[0]:
        current_location = "Prices Fork"
    elif "Bald Knob" in line[0]:
        current_location = "Bald Knob"
    if current_location and line[0] == "RECV":
        num = line[recv_format["data"]]
        locations[current_location].append(num[num.rfind("#")+1:])
print("{:<20}\t{:<20}".format("Packets Recieved", "Packets Sent"))
print("-"*40)
print("{:<20}\t{:<20}".format(len(locations["Drone Field"]), int(locations["Drone Field"][-1]) - int(locations["Drone Field"][0])+1))
print("{:<20}\t{:<20}".format(len(locations["Prices Fork"]), int(locations["Prices Fork"][-1]) - int(locations["Prices Fork"][0])+1))
print("{:<20}\t{:<20}".format(len(locations["Bald Knob"]), int(locations["Bald Knob"][-1]) - int(locations["Bald Knob"][0])+1))
miss_rate["Drone Field"] = 1-(len(locations["Drone Field"]) / (int(locations["Drone Field"][-1])-int(locations["Drone Field"][0])+1))
miss_rate["Prices Fork"] = 1-(len(locations["Prices Fork"]) / (int(locations["Prices Fork"][-1])-int(locations["Prices Fork"][0])+1))
miss_rate["Bald Knob"] = 1-(len(locations["Bald Knob"]) / (int(locations["Bald Knob"][-1])-int(locations["Bald Knob"][0])+1))
print("-"*40)
print(f"Packet Drop Rate:\n Drone Field: {miss_rate['Drone Field']:.4f} %\n Prices Fork: {miss_rate['Prices Fork']:.4f} %\n Bald Knob: {miss_rate['Bald Knob']:.4f} %")

Packets Recieved    	Packets Sent        
----------------------------------------
39                  	39                  
18                  	34                  
272                 	290                 
----------------------------------------
Packet Drop Rate:
 Drone Field: 0.0000 %
 Prices Fork: 0.4706 %
 Bald Knob: 0.0621 %


In [142]:
current_location = None
recv_format = {data[2][x]: x for x in range(len(data[2]))}
gps_format = {data[3][x]: x for x in range(len(data[3]))}
last_coords=None
for line in data[4:]:
    if "Drone Field" in line[0]:
        current_location = "Drone Field"
    elif "Prices Fork" in line[0]:
        current_location = "Prices Fork"
    elif "Bald Knob" in line[0]:
        current_location = "Bald Knob"
        
    if line[0] == "RECV":
        popup= f"SNR:{line[recv_format["SNR(db)"]]}dB\nRSSI:{line[recv_format["RSSI(dBm)"]]}dBm"
        data=line[recv_format['data']]
        tooltip = f"{current_location} #{data[data.rfind('#')+1:]}"
        coords=(float(line[recv_format["Lat"]]), float(line[recv_format["Lon"]]))
        if coords != (1000.0, 1000.0):
            folium.Marker(coords, tooltip=tooltip, popup=popup).add_to(map)
    elif line[0] == "GPS":
        coords=(float(line[gps_format["Lat"]]), float(line[gps_format["Lon"]]))
        if coords != (1000.0, 1000.0):
            # if last_coords is not None:
            #     folium.PolyLine([last_coords, coords], color="blue", weight=2.5, opacity=1).add_to(map)
            last_coords = coords
            folium.Marker(coords, popup="Periodic GPS",icon=folium.Icon(color="red")).add_to(map)

map